In [63]:
# Let's import the necessary libs
import torch
import torchvision
import collections
from torchvision.models.detection import faster_rcnn
import torchvision.models as models
torch.manual_seed(2)

import Faster_RCNN
import detection_dataset

In [64]:
vgg_16 = torchvision.models.vgg16_bn(pretrained=True)
vgg_16_features = vgg_16.features
backbone = Faster_RCNN.feature_extractor(vgg_16_features)

image = torch.randn(1, 3, 800, 800)
backbone_op = backbone(image)
print("Output shape from backbone is ", backbone_op.shape)




Output shape from backbone is  torch.Size([1, 512, 25, 25])


In [65]:
from torchvision.models.detection import rpn
rpn_layer = rpn.RPNHead(in_channels = 256, num_anchors = 9)
anchor_generator = rpn.AnchorGenerator(sizes=((128, 256, 512),), aspect_ratios=(0.5, 1, 2))

rpn_pre_nms = dict(training=2000, testing=1000)
rpn_post_nms = dict(training=2000, testing=1000)
region_proposer = rpn.RegionProposalNetwork(anchor_generator, rpn_layer, 0.5, 0.5, 512, 0.5, 
                                            rpn_pre_nms, rpn_post_nms, 0.7 )
region_proposer.training = False

from torchvision.ops import MultiScaleRoIAlign
roi_pooler = MultiScaleRoIAlign(['0'], output_size=(7, 10), sampling_ratio=2)

from torchvision.models.detection import faster_rcnn
from torchvision.models.detection import image_list
import cv2
mlp_head = faster_rcnn.TwoMLPHead(in_channels = 256*7*10, representation_size=1024)
final_layer = faster_rcnn.FastRCNNPredictor(in_channels=1024, num_classes=17)


In [66]:
model = Faster_RCNN.FasterRCNN(backbone, region_proposer, roi_pooler, mlp_head, final_layer)

In [78]:
from torch.utils.data import DataLoader

Means = [0.485, 0.456, 0.406]
Stds = [0.229, 0.224, 0.225]

train_dataset = detection_dataset.detection_Dataset(train=True,transforms = True)
test_dataset = detection_dataset.detection_Dataset(train=False, transforms = True)

def custom_collate_fn(batch):
    images = [item['image'] for item in batch]
    annotations = [item['annotation'] for item in batch]
    return {'image': images, 'annotation': annotations}


train_dataloader = DataLoader(
    train_dataset,
    batch_size =10,
    shuffle = True,
    collate_fn = custom_collate_fn
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size = 10,
    shuffle = True,
    collate_fn = custom_collate_fn
)


In [79]:
# GPU 사용 설정
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# 옵티마이저와 학습률 스케줄러 설정
optimizer = model.define_optimizer(lr=0.0001)
base_width, base_height = 800, 800

# 모델 저장
for ephoch in range(10):
    model.train()
    
    for i, data in enumerate(train_dataloader):
        images = torch.stack([image for image in data['image']])
        annotations = data['annotation']
        loss_dict = model(images, annotations)
        print(loss_dict)
        losses = sum(loss for loss in loss_dict.values())
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        print(f"Loss : {losses}")

    if ephoch % 5 == 0:
        torch.save(model.state_dict(), f"model_{ephoch}.pth")
        print(f"Model saved at epoch {ephoch}")


RuntimeError: [enforce fail at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10\core\impl\alloc_cpu.cpp:81] data. DefaultCPUAllocator: not enough memory: you tried to allocate 1638400000 bytes.